In [1]:
# written by Fernando Campos 
# fcampos@cicese.edu.mx

import xarray as xr
import numpy as np
import glob 
import xgcm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap, cm, shiftgrid
import xesmf as xe
from scipy import signal

from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=3, threads_per_worker=16)
client = Client(cluster)
from dask import delayed

In [20]:
season = "jas"
season_title = "Jul-Aug-Sep"

In [21]:
ds0 = xr.open_mfdataset("/HOME/users/fcampos/outputs/mask.nc")
dr0 = xr.open_mfdataset(
    np.hstack([glob.glob("/HOME/users/fcampos/outputs/global/Ro/"+season+"/Ro_8640x6480_2012*.nc"),
               glob.glob("/HOME/users/fcampos/outputs/global/ke/"+season+"/ke_8640x6480_2012*.nc")]),
                       parallel=True, data_vars="different")

# Gulf Stream

In [ ]:
name = "Gulf Stream"
name_res = "GS"

ds = ds0.isel(i=slice(8000,8400), j=slice(4700,5150), i_g=slice(8000,8400), j_g=slice(4700,5150))
dr = dr0.isel(i=slice(8000,8400), j=slice(4700,5150))

dr.coords["XC"] = ds.XC
dr.coords["YC"] = ds.YC
dr = dr.rename({"XC": "lon", "YC": "lat"})

ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(30, 41, 0.04)),
        "lon": (["lon"], np.arange(-60,-49, 0.04)),
    }
)
regridder = xe.Regridder(dr, ds_out, "bilinear")
ds_out = regridder(dr)
del ds, dr

# North Atlantic

In [22]:
name = "North Atlantic"
name_res = "NA"

ds = ds0.isel(i=slice(420,700), j=slice(4900,5300), i_g=slice(420,700), j_g=slice(4900,5300))
dr = dr0.isel(i=slice(420,700), j=slice(4900,5300))

dr.coords["XC"] = ds.XC
dr.coords["YC"] = ds.YC
dr = dr.rename({"XC": "lon", "YC": "lat"})

ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(34, 45, 0.04)),
        "lon": (["lon"], np.arange(-21,-10, 0.04)),
    }
)
regridder = xe.Regridder(dr, ds_out, "bilinear")
ds_out = regridder(dr)
del ds, dr

# Kuroshio Extension

In [ ]:
name = "Kuroshio Extension"
name_res = "KE"

ds = ds0.isel(i=slice(4500,4800), j=slice(4700,5200), i_g=slice(4500,4800), j_g=slice(4700,5200))
dr = dr0.isel(i=slice(4500,4800), j=slice(4700,5200))

dr.coords["XC"] = ds.XC
dr.coords["YC"] = ds.YC
dr = dr.rename({"XC": "lon", "YC": "lat"})

ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(30, 40, 0.04)),
        "lon": (["lon"], np.arange(150,160, 0.04)),
    }
)
regridder = xe.Regridder(dr, ds_out, "bilinear")
ds_out = regridder(dr)
del ds, dr

# North Pacific

In [ ]:
name = "North Pacific"
name_res = "NP"

ds = ds0.isel(i=slice(6300,6600), j=slice(4700,5200), i_g=slice(6300,6600), j_g=slice(4700,5200))
dr = dr0.isel(i=slice(6300,6600), j=slice(4700,5200))

dr.coords["XC"] = ds.XC
dr.coords["YC"] = ds.YC
dr = dr.rename({"XC": "lon", "YC": "lat"})

ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(30, 40, 0.04)),
        "lon": (["lon"], np.arange(-135,-125, 0.04)),
    }
)
regridder = xe.Regridder(dr, ds_out, "bilinear")
ds_out = regridder(dr)
del ds, dr

 # Antarctic Circumpolar Current

In [ ]:
name = "Antartic Circumpolar Current"
name_res = "ACC"

ds = ds.isel(i=slice(2000,3000), j=slice(2000,3000), i_g=slice(2000,3000), j_g=slice(2000,3000))
dr = dr.isel(i=slice(2000,3000), j=slice(2000,3000))

dr.coords["XC"] = ds.XC
dr.coords["YC"] = ds.YC
dr = dr.rename({"XC": "lon", "YC": "lat"})

ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-50, -40, 0.04)),
        "lon": (["lon"], np.arange(55,65, 0.04)),
    }
)
regridder = xe.Regridder(dr, ds_out, "bilinear")
ds_out = regridder(dr)
del ds, dr

# ===============================
# From here, start the spectral analysis
# ===============================

In [ ]:
phike = signal.detrend(ds_out.ke,axis=0,type="linear")
phike = signal.detrend(phike,axis=1,type="linear")
phike = signal.detrend(phike,axis=2,type="linear")
phizeta = signal.detrend(ds_out.vorticity,axis=0,type="linear")
phizeta = signal.detrend(phizeta,axis=2,type="linear")
phizeta = signal.detrend(phizeta,axis=2,type="linear")

dt, dx, dy = (1, 0.04*110, 0.04*110)
nt, nx, ny = ds_out.ke.shape
Lt, Lx, Ly = (dt*nt, dx*nx, dy*ny)

win1 =  np.hanning(nt)
win1 =  (nt/(win1**2).sum())*win1
win2 =  np.hanning(nx)
win2 =  (nx/(win2**2).sum())*win2
win3 =  np.hanning(ny)
win3 =  (ny/(win3**2).sum())*win3

win = win2[np.newaxis]*win1[...,np.newaxis]
win = win[...,np.newaxis]*win3[np.newaxis,np.newaxis]

phike = phike*win
phizeta = phizeta*win

dkt, dkx, dky = (1/Lt, 1/Lx, 1/Ly)

kt = np.fft.rfftfreq(nt, d=dt)
kx = np.fft.rfftfreq(nx, d=dx)
ky = np.fft.rfftfreq(ny, d=dy)

phike = np.fft.fftn(phike,axes=(2,1,0))
phizeta = np.fft.fftn(phizeta,axes=(2,1,0))

specke = (phike*phike.conj()).real/(dkt*dkx*dky)/(nt*nx*ny)**2
speczeta = (phizeta*phizeta.conj()).real/(dkt*dkx*dky)/(nt*nx*ny)**2

specke = specke[:int(np.floor(nt/2)+1),:int(np.floor(ny/2)+1),:int(np.floor(nx/2))+1]
specke[1:int(np.floor(nt/2)),1:int(np.floor(ny/2)),1:int(np.floor(nx/2))] = \
                 8*specke[1:int(np.floor(nt/2)),1:int(np.floor(ny/2)),1:int(np.floor(nx/2))]

speczeta = speczeta[:int(np.floor(nt/2)+1),:int(np.floor(ny/2)+1),:int(np.floor(nx/2))+1]
speczeta[1:int(np.floor(nt/2)),1:int(np.floor(ny/2)),1:int(np.floor(nx/2))] = \
                 8*speczeta[1:int(np.floor(nt/2)),1:int(np.floor(ny/2)),1:int(np.floor(nx/2))]

del phike, phizeta, win

kx1, ky1 = np.meshgrid(kx,ky)
kxy = np.sqrt(kx1**2+ky1**2)
dkr = np.sqrt(dkx**2 + dky**2)
if kx.max()>ky.max():
    kmax = ky.max()
else:
    kmax = kx.max() 
kr =  np.arange(dkr/2.,kmax+dkr/2.,dkr) 
Erke = np.zeros((kr.size,kt.size))
Erzeta = np.zeros((kr.size,kt.size))

def compute_Er(spec,kxy,kr,dkr):
    fkr = (kxy>=kr-dkr/2) & (kxy<=kr+dkr/2)        
    dth = np.pi / (fkr.sum()-1)
    Er = (spec*fkr*kxy*dth).sum(axis=(1,2)) 
    return Er

for i in range(kr.size):
    Erke[i,:] = compute_Er(specke,kxy,kr[i],dkr)    
for i in range(kr.size):
    Erzeta[i,:] = compute_Er(speczeta,kxy,kr[i],dkr)

del specke, speczeta, kxy 

kkt, kkr = np.meshgrid(kt,kr)
Erke=Erke*kkt*kkr
Erzeta=Erzeta*kkt*kkr

2022-11-20 21:09:54,566 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2022-11-20 21:10:04,043 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2022-11-20 21:10:06,425 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2022-11-20 21:10:20,273 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2022-11-20 21:10:29,136 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize=(8,7), constrained_layout=True)

ax1.loglog()
az1=ax1.pcolormesh(kkr,kkt,Erke,norm=colors.LogNorm(vmin=1e-5,vmax=5e-2),cmap="rainbow")
ax1.set_xlim(1e-3,1e-1)
ax1.set_ylim(1e-3,kt.max())
cb = plt.colorbar(az1, ax=ax1)
ax1.set_title(str(name)+" \n $\omega$-k spectrum for KE 2012 ("+str(season_title)+")", fontsize=18)
ax1.set_xlabel("horizontal wavenumber [cpkm]", fontweight="bold")
ax1.set_ylabel("Frequency [cph]", fontweight="bold")
ax1.tick_params(axis="both", labelsize=16)
cb.set_label(label='', size='large', weight='bold')

plt.savefig("isospectrum_2012_"+str(name_res)+"_ke_"+str(season)+".png", dpi=200)

In [ ]:
fig, (ax2) = plt.subplots(1,1, figsize=(8,7), constrained_layout=True)

ax2.loglog()
az2=ax2.pcolormesh(kkr,kkt,Erzeta,norm=colors.LogNorm(vmin=1e-5,vmax=5e-1),cmap="turbo")
ax2.set_xlim(1e-3,1e-1)
ax2.set_ylim(1e-3,kt.max())
plt.colorbar(az2, ax=ax2)
ax2.set_title(str(name)+" \n $\omega$-k spectrum for Ro 2012 ("+str(season_title)+")", fontsize=18)
ax1.set_ylabel("Frequency [cph]", fontweight="bold")
ax2.set_xlabel("horizontal wavenumber [cpkm]", fontweight="bold")
ax2.tick_params(axis="both", labelsize=16)

plt.savefig("isospectrum_2012_"+str(name_res)+"_ro_"+str(season)+".png", dpi=200)